In [2]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import os
from tqdm import tqdm

In [4]:
npfile = np.load(r'G:\Mi unidad\Maestria\Proyecto de Grado\analisis_nf\datos\australia\austotalyr_DTW.npz')
print(npfile.files)

['lat', 'lon', 'HydroID', 'dtw_syr', 'dtw_condyr', 'dtw_s_nmax_yr', 'dtw_nmax_yr', 'dtw_vyr', 'dtw_tyr', 'dtw_resyr']


In [7]:
variables = {
    'dtw_syr': npfile['dtw_syr'],
    'dtw_condyr': npfile['dtw_condyr'],
    'dtw_s_nmax_yr': npfile['dtw_s_nmax_yr'],
    'dtw_nmax_yr': npfile['dtw_nmax_yr'],
    'dtw_tyr': npfile['dtw_tyr'],
    'dtw_vyr': npfile['dtw_vyr'],
    'HydroID': npfile['HydroID'],
    'lat': npfile['lat'],
    'lon': npfile['lon']
}

# Crear array de coordenadas (lon, lat)
coords = np.column_stack((variables['lon'], variables['lat']))

In [8]:
# 2. Cargar y preparar shapefile
shp_path = r'G:\Mi unidad\Maestria\Proyecto de Grado\analisis_nf\datos\ARTÍCULO\australia_aquifers.shp'
mapa = gpd.read_file(shp_path).to_crs(epsg=3577)  # CRS proyectado para Australia
mapa = mapa[mapa['Broader'] != '-']  # Filtramos cuencas válidas
mapa['area'] = mapa.geometry.area  # Calculamos áreas
mapa = mapa.to_crs(epsg=4326)  # Volvemos a WGS84

In [9]:
# 3. Pre-allocar diccionario para resultados
cuencas_validas = mapa['Broader'].unique()
resultados = {cuenca: {var: [] for var in variables.keys()} for cuenca in cuencas_validas}
resultados['indices'] = {cuenca: [] for cuenca in cuencas_validas}

In [10]:
# 4. Asignación optimizada (usando índices)
puntos = [Point(lon, lat) for lon, lat in zip(variables['lon'], variables['lat'])]

for idx, (nombre, geometria) in tqdm(enumerate(zip(mapa['Broader'], mapa.geometry)), 
                                  total=len(mapa),
                                  desc="Asignando pozos a cuencas"):
    # Máscara vectorizada
    dentro = np.array([geometria.contains(p) for p in puntos], dtype=bool)
    indices = np.where(dentro)[0]
    
    if len(indices) > 0:
        resultados['indices'][nombre].extend(indices)

# 5. Agrupar datos y guardar NPZ por cuenca
output_dir = r'G:\Mi unidad\Maestria\Proyecto de Grado\analisis_nf\datos\australia\cuencas_hidrogeologicas'
os.makedirs(output_dir, exist_ok=True)

for cuenca, indices in tqdm(resultados['indices'].items(), desc="Guardando archivos"):
    if indices:
        idx_array = np.array(indices, dtype=np.int32)
        
        # Crear diccionario con todos los datos filtrados
        datos_cuenca = {var: variables[var][idx_array] for var in variables.keys()}
        
        # Añadir metadatos
        datos_cuenca['cuenca'] = cuenca
        datos_cuenca['num_pozos'] = len(idx_array)
        
        # Guardar con compresión
        nombre_archivo = f"{cuenca.lower().replace(' ', '_').replace('/', '_')}.npz"
        np.savez_compressed(
            os.path.join(output_dir, nombre_archivo),
            **datos_cuenca
        )

print("Proceso completado. Archivos guardados en:", output_dir)

Guardando archivos: 100%|██████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 11.33it/s]

Proceso completado. Archivos guardados en: G:\Mi unidad\Maestria\Proyecto de Grado\analisis_nf\datos\australia\cuencas_hidrogeologicas
